In [2]:
# import modules for math and distributions
from math import exp
import numpy as np
from scipy.stats import gamma, norm, uniform, bernoulli

In [3]:
# define inverse logit function
def logit_1(x): return(exp(x)/(1 + exp(x)))

In [4]:
# Initial set up according to HD2012
T = 40 # time periods
k = 5 # check-up times
theta = [-0.405, 0.0205, -0.00405]
gam = [-3, 0.005, -1.5, 0.1]

In [5]:
# define lists for holding A, L, U and Y
A = np.zeros(T + 1) # A[-1] holds the value for A in t = -1
L = np.zeros(T)
U = np.zeros(T)
Y = np.zeros(T + 1)
eps = np.zeros(T)
lam = np.zeros(T) # prob of failure at each time period
delta = np.zeros(T)

# set the first value of U, U[0], to a 
# randomly generated value from a uniform
# distribution a measure of general health
U[0] = uniform.rvs()
eps[0] = norm.rvs(0, 20)
L[0] = gamma.ppf(U[0], 3, 154)

# set A[-1] to 0
A[-1] = 0
x = logit_1(theta[0] + theta[2] * (L[0] - 500))
A[0] = bernoulli.rvs(x, size=1)

In [6]:
if A[0] == 1: Ts = 0

In [7]:
lam[0] = logit_1(gam[0] + gam[2] * A[0])

In [8]:
if lam[0] >= U[0]:
    Y[1] = 1
else:
    Y[1] = 0

In [9]:
# loop for each individual
for t in range(1, T):
    if Y[t] == 0:
        delta[t] = norm.rvs(0, 0.05)
        U[t] = min(1, max(0, U[t-1] + delta[t]))
        if t % k != 0:
            L[t] = L[t-1]
            A[t] = A[t-1]
        else:
            eps[1] = norm.rvs(100 * (U[t] - 2), 50)
            L[t] = max(0, L[t-1] + 150 * A[t-k] * (A[t-k-1]) + eps[t])
            if A[t-1] == 0:
                A[t] = bernoulli.rvs(logit_1(theta[0] + theta[1] * t + theta[2] * (L[t] - 500)), size=1)
            else:
                A[t] = 1
            if A[t] == 1 and A[t-k] == 0: Ts = t
        lam[t] = logit_1(gam[0] + gam[1] * ((1 - A[t]) * t + A[t] * Ts) + gam[2] * A[t] + gam[3] * A[t] *(t - Ts))
        if (1 - np.prod(1 - lam)) >= U[0]:
            Y[t + 1] = 1
        else:
            Y[t + 1] = 0
    else:
        break

     

In [52]:
# make a function that does all of the above for an individual
def sim(T, k, gam, theta):
    
    # define lists for holding A, L, U and Y
    A = np.zeros(T + 1) # A[-1] holds the value for A in t = -1
    L = np.zeros(T)
    U = np.zeros(T)
    Y = np.zeros(T + 1)
    eps = np.zeros(T)
    lam = np.zeros(T) # prob of failure at each time period
    delta = np.zeros(T)

    # set the first value of U, U[0], to a 
    # randomly generated value from a uniform
    # distribution a measure of general health
    U[0] = uniform.rvs()
    eps[0] = norm.rvs(0, 20)
    L[0] = gamma.ppf(U[0], 3, 154)

    # set A[-1] to 0
    A[-1] = 0
    x = logit_1(theta[0] + theta[2] * (L[0] - 500))
    A[0] = bernoulli.rvs(x, size=1)
    
    # if A[0] == 1: Ts = 0 check
    Ts = 0
    
    lam[0] = logit_1(gam[0] + gam[2] * A[0])
    
    if lam[0] >= U[0]:
        Y[1] = 1
    else:
        Y[1] = 0
    # loop through each time period - stop when patient is dead or t = T + 1
    for t in range(1, T):
        if Y[t] == 0:
            delta[t] = norm.rvs(0, 0.05)
            U[t] = min(1, max(0, U[t-1] + delta[t]))
            if t % k != 0:
                L[t] = L[t-1]
                A[t] = A[t-1]
            else:
                eps[1] = norm.rvs(100 * (U[t] - 2), 50)
                L[t] = max(0, L[t-1] + 150 * A[t-k] * (A[t-k-1]) + eps[t])
                if A[t-1] == 0:
                    A[t] = bernoulli.rvs(logit_1(theta[0] + theta[1] * t + theta[2] * (L[t] - 500)), size=1)
                else:
                    A[t] = 1
                if A[t] == 1 and A[t-k] == 0: Ts = t
            lam[t] = logit_1(gam[0] + gam[1] * ((1 - A[t]) * t + A[t] * Ts) + gam[2] * A[t] + gam[3] * A[t] *(t - Ts))
            if (1 - np.prod(1 - lam)) >= U[0]:
                Y[t + 1] = 1
            else:
                Y[t + 1] = 0
        else:
            break
    
    # we only need the data before death, so whatever value t is before the end of the
    # above loop
    Y = np.ndarray.tolist(Y[0:t])
    U = np.ndarray.tolist(U[0:t])
    L = np.ndarray.tolist(L[0:t])
    A = np.ndarray.tolist(A[0:t])
    #Ts = [Ts]*t

    return({"Y":Y, "U":U, "L":L, "A":A, "Ts":Ts, "time":[time for time in range(t)]})

In [53]:
import pandas as pd
n = 1000
d = sim(T, k, gam, theta)
frames = [pd.DataFrame.from_dict(d) for i in range(n)]
result = pd.concat(frames)
